In [26]:
# Define the finite field
F = GF(int("c90102faa48f18b5eac1f76bb40a1b9fb0d841712bbe3e5576a7a56976c2baeca47809765283aa078583e1e65172a3fd", 16))

# Define the elliptic curve parameters
a = F(int("a079db08ea2470350c182487b50f7707dd46a58a1d160ff79297dcc9bfad6cfc96a81c4a97564118a40331fe0fc1327f", 16))  # coefficient a
b = F(int("9f939c02a7bd7fc263a4cce416f4c575f28d0c1315c4f0c282fca6709a5f9f7f9c251c9eede9eb1baa31602167fa5380", 16))  # coefficient b

# Create the elliptic curve
E = EllipticCurve(F, [a, b])

# Print information about the curve
print(f"Elliptic curve: y^2 = x^3 + {a}x + {b} over GF({p})")
print(f"Order of the curve: {E.order()}")
print(f"Generator point: {E.gens()[0]}")

Elliptic curve: y^2 = x^3 + 24699516740398840043612817898240834783822030109296416539052220535505263407290501127985941395251981432741860384780927x + 24561086537518854907476957344600899117700350970429030091546712823181765905950742731855058586986320754303922826007424 over GF(30937339651019945892244794266256713890440922455872051984768764821736576084296075471241474533335191134590995377857533)
Order of the curve: 30937339651019945892244794266256713890440922455872051984762505561763526780311616863989511376879697740787911484829297
Generator point: (11388440216840225867914685855101918818479484574596379294321037492678209786348438087554628192600027392965456023758684 : 13855220083453625433835136988507326025523893392221309217843727317241632693088693242352993636002292915507051963872681 : 1)


In [16]:
print("Generator x", hex(11388440216840225867914685855101918818479484574596379294321037492678209786348438087554628192600027392965456023758684))
print("Generator x", hex(13855220083453625433835136988507326025523893392221309217843727317241632693088693242352993636002292915507051963872681))

Generator x 0x49fe001bbdbd3942e27885e83fe3d7cfb3a390e2190513e49c4e671d70d1ce111c88c0f1783f1e3ebed81d0addd7a75c
Generator x 0x5a04e911a8cdc5c55485e4344282c84efe1c43b84004bd702634a6c90a0a7d77c76f133067619cb500c38bbebcfbf5a9


In [30]:
client_gen_x = int("087b5fe3ae6dcfb0e074b40f6208c8f6de4f4f0679d6933796d3b9bd659704fb85452f041fff14cf0e9aa7e45544f9d8", 16)
client_gen_y = int("127425c1d330ed537663e87459eaa1b1b53edfe305f6a79b184b3180033aab190eb9aa003e02e9dbf6d593c5e3b08182", 16)
print("Client Gen", client_gen_x, client_gen_y)
print(E.is_on_curve(client_gen_x, client_gen_y))

client_x = int("195b46a760ed5a425dadcab37945867056d3e1a50124fffab78651193cea7758d4d590bed4f5f62d4a291270f1dcf499", 16)
client_y = int("357731edebf0745d081033a668b58aaa51fa0b4fc02cd64c7e8668a016f0ec1317fcac24d8ec9f3e75167077561e2a15", 16)
print("Client Point", client_x, client_y)
print(E.is_on_curve(client_x, client_y))

server_x = int("b3e5f89f04d49834de312110ae05f0649b3f0bbe2987304fc4ec2f46d6f036f1a897807c4e693e0bb5cd9ac8a8005f06", 16)
server_y = int("85944d98396918741316cd0109929cb706af0cca1eaf378219c5286bdc21e979210390573e3047645e1969bdbcb667eb", 16)
print("Server Point", server_x, server_y)
print(E.is_on_curve(server_x, server_y))

Client Gen 1305488802776637960515697387274764814560693662216913070824404729088258519836180992623611650289275235949409735080408 2840284555446760004012395483787208388204705580027573689198385753943125520419959469842139003551394700125370894549378
True
Client Point 3902729749136290727021456713077352817203141198354795319199240365158569738643238197536678384836705278431794896368793 8229109857867260486993831343979405488668387983876094644791511977475828392446562276759399366591204626781463052691989
True
Server Point 27688886377906486650974531457404629460190402224453915053124314392088359043897605198852944594715826578852025617899270 20559737347380095279889465811846526151405412593746438076456912255094261907312918087801679069004409625818172174526443
True
